# In order to use dask
1. Activate Conda Python 3 `source activate python3`
2. Install fastparquet `conda install -c conda-forge fastparquet'
3. Restart notebook kernal ensuring you are on conda_python3 environment

In [13]:
import boto3
import dask.dataframe as dd
from sagemaker import get_execution_role
from fastparquet import ParquetFile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

role = get_execution_role()
bucket='tally-ai-dspt3'
folder = 'yelp-kaggle-raw-data'

In [14]:
#Loading data
data = 'final_combined.parquet.gzip'
data_location = 's3://{}/{}/{}'.format(bucket, folder, data)
df = dd.read_parquet(data_location)
df.head()


,address,business_id,categories,city,is_open,latitude,longitude,postal_code,state,stars_business,...,compliment_writer,elite,fans,friends,yelping_since,name_users,review_count_users,cool_users,funny_users,useful_users
index,,,,,,,,,,,,,,,,,,,,,
0,"3175 Rutherford Rd, Unit 47",E7v9sMJx4_Swx7VtymNt5Q,"Food Delivery Services, Food, Thai, Restaurants",Vaughan,1,43.827830,-79.538072,L4K 5Y6,ON,4.0,...,0,,0,None,2011-06-22 21:48:15,F,4,2,0,6
1,2777 Steeles Avenue W,4Lh1lZXTf0EoNqdzT60GPQ,"Coffee & Tea, Food, Ice Cream & Frozen Yogurt,...",Toronto,0,43.782160,-79.490430,M3J 3K5,ON,4.0,...,0,,0,None,2011-06-22 21:48:15,F,4,2,0,6
2,11155 S Eastern Ave,RPjBB_uhHQ4oxp39Jqm_HQ,"Sandwiches, Restaurants",Henderson,1,35.989803,-115.101597,89052,NV,3.0,...,0,,1,"CyOjizbfMb2hxkLIwE76Ow, MbZ0W9PjmlI7J9fs4SiNrw...",2013-02-08 21:54:49,Jake,50,17,17,53
3,"140 S Green Valley Pkwy, Ste 142",SR0Q2hAEMJ_m23O_x-khSQ,"Restaurants, American (New), Gluten-Free, Vege...",Henderson,0,36.022441,-115.083539,89012,NV,4.0,...,0,,1,"CyOjizbfMb2hxkLIwE76Ow, MbZ0W9PjmlI7J9fs4SiNrw...",2013-02-08 21:54:49,Jake,50,17,17,53
4,"8174 S Las Vegas Blvd, Ste 101",V_TxEdzeEs8P5nGPANOY7A,"Cocktail Bars, Gay Bars, Cafes, Bars, Nightlif...",Las Vegas,0,36.040642,-115.170643,89123,NV,3.5,...,0,,1,"CyOjizbfMb2hxkLIwE76Ow, MbZ0W9PjmlI7J9fs4SiNrw...",2013-02-08 21:54:49,Jake,50,17,17,53


In [15]:
#Convert dd to pd dataframe
df = df.compute()

In [16]:
df['datetime'] = pd.to_datetime(df['date'])
df['datetime'].describe()
# the timespan is about 14 years

count                 5055992
unique                4988798
top       2014-07-28 05:36:33
freq                        7
first     2004-10-12 10:13:32
last      2019-12-13 15:50:49
Name: datetime, dtype: object

In [17]:
#need name
buisness_col = ['business_id', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars_review', 'review_count_reviews', 'categories']
rev_col = ['review_id', 'business_id', 'stars_review', 'datetime', 'text']
analytical_col = ['review_id', 'word_lenght']

In [18]:
biz = pd.DataFrame(df, columns = buisness_col)
biz.head()

,business_id,address,city,state,postal_code,latitude,longitude,stars_review,review_count_reviews,categories
index,,,,,,,,,,
0,E7v9sMJx4_Swx7VtymNt5Q,"3175 Rutherford Rd, Unit 47",Vaughan,ON,L4K 5Y6,43.827830,-79.538072,5,125,"Food Delivery Services, Food, Thai, Restaurants"
1,4Lh1lZXTf0EoNqdzT60GPQ,2777 Steeles Avenue W,Toronto,ON,M3J 3K5,43.782160,-79.490430,4,22,"Coffee & Tea, Food, Ice Cream & Frozen Yogurt,..."
2,RPjBB_uhHQ4oxp39Jqm_HQ,11155 S Eastern Ave,Henderson,NV,89052,35.989803,-115.101597,2,95,"Sandwiches, Restaurants"
3,SR0Q2hAEMJ_m23O_x-khSQ,"140 S Green Valley Pkwy, Ste 142",Henderson,NV,89012,36.022441,-115.083539,4,525,"Restaurants, American (New), Gluten-Free, Vege..."
4,V_TxEdzeEs8P5nGPANOY7A,"8174 S Las Vegas Blvd, Ste 101",Las Vegas,NV,89123,36.040642,-115.170643,5,127,"Cocktail Bars, Gay Bars, Cafes, Bars, Nightlif..."


In [19]:
biz.shape

(5055992, 10)

In [20]:
data = 'biz-onepart'
folder = 'biz-rev-analytic'
bucket = 'tally-ai-dspt3'
data_location = 's3://{}/{}/{}'.format(bucket, folder, data)

In [21]:
print(data_location)

s3://tally-ai-dspt3/biz-rev-analytic/biz-onepart


In [22]:
#previously did 5 partitions
#now trying 1 partition
biz = dd.from_pandas(biz, npartitions=1)

In [24]:
# directly save to S3 bucket
# make utility function to load and save data



dd.to_csv(biz, 's3://tally-ai-dspt3/biz-rev-analytic/biz-one-part')

['tally-ai-dspt3/biz-rev-analytic/biz/0.part']

In [ ]:
#read 
biz = dd.read_csv('s3://tally-ai-dspt3/biz-rev-analytic/rev.csv') 

In [12]:
rev = pd.DataFrame(df, columns = rev_col)
rev.head()

,review_id,business_id,stars_review,datetime,text
index,,,,,
0,TqVu_UhFlbrX7N-NFm3Kwg,E7v9sMJx4_Swx7VtymNt5Q,5,2011-06-23 00:52:30,"Had duck curry, coconut rice and Thai style pa..."
1,he__NcybQ58xBqOjd3DCaw,4Lh1lZXTf0EoNqdzT60GPQ,4,2016-01-19 23:24:55,First time here (thanks Groupon!). Big burger...
2,_uXQHRbF7eygy7eVk268tg,RPjBB_uhHQ4oxp39Jqm_HQ,2,2014-02-19 20:40:15,Hate to do this to this place but as I write t...
3,BO7NZvQ8sXDfxyiaOhMa1A,SR0Q2hAEMJ_m23O_x-khSQ,4,2014-07-15 18:24:45,"First off I am giving an extra star for clean,..."
4,-Q7K1WVpM-aBaVB-8PEoQA,V_TxEdzeEs8P5nGPANOY7A,5,2015-03-19 12:40:11,Went in here for lunch yesterday to try it out...


In [14]:
rev = dd.from_pandas(rev, npartitions=5)

TypeError: Input must be a pandas DataFrame or Series

In [15]:
# directly save to S3 bucket
# make utility function to load and save data



dd.to_csv(rev, 's3://tally-ai-dspt3/biz-rev-analytic/rev.csv')

['tally-ai-dspt3/biz-rev-analytic/rev.csv/0.part',
 'tally-ai-dspt3/biz-rev-analytic/rev.csv/1.part',
 'tally-ai-dspt3/biz-rev-analytic/rev.csv/2.part',
 'tally-ai-dspt3/biz-rev-analytic/rev.csv/3.part',
 'tally-ai-dspt3/biz-rev-analytic/rev.csv/4.part']

In [ ]:
rev = dd.read_csv('taally_S3_bucketlink', np)

In [ ]:
rev = dd.read_csv('s3://tally-ai-dspt3/biz-rev-analytic/rev.csv') 

# Vader

In [ ]:
#traaining job or python script in background (could turn off instance)

## Gensim and Vader need to be installed

# In order to use Gensim
1. Activate Conda Python 3 `source activate python3`
2. Install Gensim `conda install -c conda-forge gensim'
3. Restart notebook kernal ensuring you are on conda_python3 environment

# In order to use Vader
1. pip install vaderSentiment

In [16]:
import re
import nltk
import json
import numpy as np
import pandas as pd
from nltk.corpus import wordnet 
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

# from tallylib.sql import getLatestReviews # for django app

nltk.download('wordnet')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_colwidth', 1000)


from google.colab import drive
drive.mount("/content/drive", force_remount=True)

ModuleNotFoundError: No module named 'gensim'

In [ ]:
df = pd.read_csv('/content/drive/My Drive/cafesFromDB.csv')
#command ran in database
# select * from tallyds.yelp_review where text like '%cafe%' order by business_id limit 500;

df = df[df['business_id'] == '0d6kx6Jlocw77y1J9nbqMA']# just work with one popular cafe

def tokenizer(doc):
     return [token for token in simple_preprocess(doc) 
             if token not in STOPWORDS]


def related_to_food(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('food.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_service(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('service.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_speed(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('speed.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_price(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('price.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_ambience(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('ambience.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list


def related_to_experience(doc):
  word_similarity_list = []
  for review_word in doc:
    syns = wordnet.synsets(review_word) 
    if len(syns) > 0:
      w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
      w2 = wordnet.synset('experience.n.01') 
      word_similarity_score = w1.wup_similarity(w2)
      if word_similarity_score !=None and word_similarity_score > 0.5:
        word_similarity_list.append(review_word)
  return word_similarity_list

def extract_subject_related_words():
  df['text'] = df['text'].apply(lambda x:" ".join(re.findall("[a-zA-Z]+", x)))
  df['cleaned'] = df['text'].apply(tokenizer)
  df['words_related_to_food'] = df['cleaned'].apply(related_to_food)
  df['words_related_to_service'] = df['cleaned'].apply(related_to_service)
  df['words_related_to_speed'] = df['cleaned'].apply(related_to_speed)
  df['words_related_to_price'] = df['cleaned'].apply(related_to_price)
  df['words_related_to_ambience'] = df['cleaned'].apply(related_to_ambience)
  df['words_related_to_experience'] = df['cleaned'].apply(related_to_experience)

extract_subject_related_words()
df.sample(1)


In [ ]:
# df = df[(df['words_related_to_food'].map(len) > 1) | (df['words_related_to_service'].map(len) > 1) | (df['words_related_to_speed'].map(len) > 1) | (df['words_related_to_price'].map(len) > 1) | (df['words_related_to_ambience'].map(len) > 1) | (df['words_related_to_experience'].map(len) > 1)]

In [ ]:
def get_list_reviews_containing_subject():
  food_review_list = df[df['words_related_to_food'].map(len) > 1]['text'].tolist()
  service_review_list = df[df['words_related_to_service'].map(len) > 1]['text'].tolist()
  speed_review_list = df[df['words_related_to_speed'].map(len) > 1]['text'].tolist()
  price_review_list = df[df['words_related_to_price'].map(len) > 1]['text'].tolist()
  ambience_review_list = df[df['words_related_to_ambience'].map(len) > 1]['text'].tolist()
  experience_review_list = df[df['words_related_to_experience'].map(len) > 1]['text'].tolist()
  return food_review_list, service_review_list, speed_review_list, price_review_list, ambience_review_list, experience_review_list

food_review_list, service_review_list, speed_review_list, price_review_list, ambience_review_list, experience_review_list = get_list_reviews_containing_subject()
  # food_review_list = df[df['words_related_to_food'].map(len) > 1]['text'].tolist()
  # service_review_list = df[df['words_related_to_service'].map(len) > 1]['text'].tolist()
  # speed_review_list = df[df['words_related_to_speed'].map(len) > 1]['text'].tolist()
  # price_review_list = df[df['words_related_to_price'].map(len) > 1]['text'].tolist()
  # ambience_review_list = df[df['words_related_to_ambience'].map(len) > 1]['text'].tolist()
  # experience_review_list = df[df['words_related_to_experience'].map(len) > 1]['text'].tolist()

# Get Sentiment Score (also very slow here) 
## Make EC2 instance and see what happens 

In [ ]:
def sentiment_score(sentence):
    # Create a SentimentIntensityAnalyzer object. 
    sid_obj = SentimentIntensityAnalyzer()

    # polarity_scores method of SentimentIntensityAnalyzer oject gives a sentiment dictionary. which contains pos, neg, neu, and compound scores. 
    sentiment_dict = sid_obj.polarity_scores(sentence)

    return sentiment_dict

def get_sentiment(review_list):
    all_sentiments = []
    compounds = []

    if len(review_list) > 0:
        for review in review_list:
            score = sentiment_score(review)
            all_sentiments.append(score)

    if len(all_sentiments) > 0:
        for sentiment_dict in all_sentiments:
            compound = sentiment_dict['compound']
            compounds.append(compound)

    if len(compounds) > 0:
        avg_sentiment = sum(compounds) / len(compounds)
    
    else:
        avg_sentiment = None

    return avg_sentiment

In [ ]:
def get_scores():
  if len(food_review_list) > 0:
    food_sentiment_score = round((get_sentiment(food_review_list))*150)
  else: 
    food_sentiment_score = 75
  if len(service_review_list) > 0:
    service_sentiment_score = round((get_sentiment(service_review_list))*150)
  else: 
    service_sentiment_score = 75
  if len(speed_review_list) > 0:
    speed_sentiment_score = round((get_sentiment(speed_review_list))*150)
  else: 
    speed_sentiment_score = 75
  if len(price_review_list) > 0:
    price_sentiment_score = round((get_sentiment(price_review_list))*150)
  else: 
    price_sentiment_score = 75
  if len(ambience_review_list) > 0:
    ambience_sentiment_score = round((get_sentiment(ambience_review_list))*150)
  else: 
    ambience_sentiment_score = 75
  if len(experience_review_list) > 0:
    experience_sentiment_score = round((get_sentiment(experience_review_list))*150)
  else: 
    experience_sentiment_score = 75
  return food_sentiment_score, service_sentiment_score, speed_sentiment_score, price_sentiment_score, ambience_sentiment_score, experience_sentiment_score


food_sentiment_score, service_sentiment_score, speed_sentiment_score, price_sentiment_score, ambience_sentiment_score, experience_sentiment_score = get_scores()

In [ ]:
results = json.dumps([
        { 'subject': 'Food', 'data1': food_sentiment_score, 'data2': 0, 'maxValue': 150 },
        { 'subject': 'Service', 'data1': service_sentiment_score, 'data2': 0, 'maxValue': 150 },
        { 'subject': 'Speed', 'data1': speed_sentiment_score, 'data2': 0, 'maxValue': 150 },
        { 'subject': 'Price', 'data1': price_sentiment_score, 'data2': 0, 'maxValue': 150 },
        { 'subject': 'Ambience', 'data1': ambience_sentiment_score, 'data2': 0, 'maxValue': 150},
        { 'subject': 'Experience', 'data1': experience_sentiment_score, 'data2': 0, 'maxValue': 150}
])
results

In [ ]:
# comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')# region can be any choose one close to you below
# # https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/Concepts.RegionsAndAvailabilityZones.html
                
# text = l7
# print(json.dumps(comprehend.batch_detect_sentiment(TextList=text, LanguageCode='en'), sort_keys=True, indent=4))

In [ ]:

# import re
# import nltk
# import json
# import numpy as np
# import pandas as pd
# from nltk.corpus import wordnet 
# from gensim.utils import simple_preprocess
# from gensim.parsing.preprocessing import STOPWORDS
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

# from tallylib.sql import getLatestReviews


# def yelpReviewSentiment(business_id):
# 	data = getLatestReviews(business_id, limit=200)
# 	if len(data)==0:
# 		return {}
# 	df = pd.DataFrame(data, columns=['date', 'text', 'stars'])
# 	del data

df = pd.read_csv('/content/drive/My Drive/cafesFromDB.csv')
#command ran in database
# select * from tallyds.yelp_review where text like '%cafe%' order by business_id limit 500;

df = df[df['business_id'] == '0d6kx6Jlocw77y1J9nbqMA']# just work with one popular cafe


def tokenizer(doc):
	return [token for token in simple_preprocess(doc) 
			if token not in STOPWORDS]


def related_to_food(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('food.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list


def related_to_service(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('service.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list

def related_to_speed(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('wait.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list


def related_to_price(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('price.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list


def related_to_ambience(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('ambience.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list


def related_to_experience(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('experience.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list

def extract_subject_related_words():
	df['text'] = df['text'].apply(lambda x:" ".join(re.findall("[a-zA-Z]+", x)))
	df['cleaned'] = df['text'].apply(tokenizer)
	df['words_related_to_food'] = df['cleaned'].apply(related_to_food)
	df['words_related_to_service'] = df['cleaned'].apply(related_to_service)
	df['words_related_to_speed'] = df['cleaned'].apply(related_to_speed)
	df['words_related_to_price'] = df['cleaned'].apply(related_to_price)
	df['words_related_to_ambience'] = df['cleaned'].apply(related_to_ambience)
	df['words_related_to_experience'] = df['cleaned'].apply(related_to_experience)


def get_list_reviews_containing_subject():
	food_review_list = df[df['words_related_to_food'].map(len) > 1]['text'].tolist()
	service_review_list = df[df['words_related_to_service'].map(len) > 1]['text'].tolist()
	speed_review_list = df[df['words_related_to_speed'].map(len) > 1]['text'].tolist()
	price_review_list = df[df['words_related_to_price'].map(len) > 1]['text'].tolist()
	ambience_review_list = df[df['words_related_to_ambience'].map(len) > 1]['text'].tolist()
	experience_review_list = df[df['words_related_to_experience'].map(len) > 1]['text'].tolist()
	return food_review_list, service_review_list, speed_review_list, price_review_list, ambience_review_list, experience_review_list



def sentiment_score(sentence):
	# Create a SentimentIntensityAnalyzer object. 
	sid_obj = SentimentIntensityAnalyzer()
	# polarity_scores method of SentimentIntensityAnalyzer oject gives a sentiment dictionary. which contains pos, neg, neu, and compound scores. 
	sentiment_dict = sid_obj.polarity_scores(sentence)

	return sentiment_dict


def get_sentiment(review_list):
	all_sentiments = []
	compounds = []

	if len(review_list) > 0:
		for review in review_list:
			score = sentiment_score(review)
			all_sentiments.append(score)

	if len(all_sentiments) > 0:
		for sentiment_dict in all_sentiments:
			compound = sentiment_dict['compound']
			compounds.append(compound)

	if len(compounds) > 0:
		avg_sentiment = sum(compounds) / len(compounds)
	
	else:
		avg_sentiment = None

	return avg_sentiment


# # tallylib/sentiment.py

# import re
# import nltk
# import json
# import numpy as np
# import pandas as pd
# from nltk.corpus import wordnet 
# from gensim.utils import simple_preprocess
# from gensim.parsing.preprocessing import STOPWORDS
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

# from tallylib.sql import getLatestReviews


# def yelpReviewSentiment(business_id):
# 	data = getLatestReviews(business_id, limit=200)
# 	if len(data)==0:
# 		return {}
# 	df = pd.DataFrame(data, columns=['date', 'text', 'stars'])
# 	del data

df = pd.read_csv('/content/drive/My Drive/cafesFromDB.csv')
#command ran in database
# select * from tallyds.yelp_review where text like '%cafe%' order by business_id limit 500;

df = df[df['business_id'] == '0d6kx6Jlocw77y1J9nbqMA']# just work with one popular cafe


def tokenizer(doc):
	return [token for token in simple_preprocess(doc) 
			if token not in STOPWORDS]


def related_to_food(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('food.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list


def related_to_service(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('service.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list


def related_to_speed(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('wait.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list


def related_to_price(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('price.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list


def related_to_ambience(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('ambience.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list


def related_to_experience(doc):
	word_similarity_list = []
	for review_word in doc:
		syns = wordnet.synsets(review_word) 
		if len(syns) > 0:
			w1 = wordnet.synset(syns[0].name()) # n here denotes the tag noun
			w2 = wordnet.synset('experience.n.01') 
			word_similarity_score = w1.wup_similarity(w2)
			if word_similarity_score !=None and word_similarity_score > 0.5:
				word_similarity_list.append(review_word)
	return word_similarity_list


def extract_subject_related_words():
	df['text'] = df['text'].apply(lambda x:" ".join(re.findall("[a-zA-Z]+", x)))
	df['cleaned'] = df['text'].apply(tokenizer)
	df['words_related_to_food'] = df['cleaned'].apply(related_to_food)
	df['words_related_to_service'] = df['cleaned'].apply(related_to_service)
	df['words_related_to_speed'] = df['cleaned'].apply(related_to_speed)
	df['words_related_to_price'] = df['cleaned'].apply(related_to_price)
	df['words_related_to_ambience'] = df['cleaned'].apply(related_to_ambience)
	df['words_related_to_experience'] = df['cleaned'].apply(related_to_experience)


def get_list_reviews_containing_subject():
	food_review_list = df[df['words_related_to_food'].map(len) > 1]['text'].tolist()
	service_review_list = df[df['words_related_to_service'].map(len) > 1]['text'].tolist()
	speed_review_list = df[df['words_related_to_speed'].map(len) > 1]['text'].tolist()
	price_review_list = df[df['words_related_to_price'].map(len) > 1]['text'].tolist()
	ambience_review_list = df[df['words_related_to_ambience'].map(len) > 1]['text'].tolist()
	experience_review_list = df[df['words_related_to_experience'].map(len) > 1]['text'].tolist()
	return food_review_list, service_review_list, speed_review_list, price_review_list, ambience_review_list, experience_review_list



def sentiment_score(sentence):
	# Create a SentimentIntensityAnalyzer object. 
	sid_obj = SentimentIntensityAnalyzer()
	# polarity_scores method of SentimentIntensityAnalyzer oject gives a sentiment dictionary. which contains pos, neg, neu, and compound scores. 
	sentiment_dict = sid_obj.polarity_scores(sentence)

	return sentiment_dict


def get_sentiment(review_list):
	all_sentiments = []
	compounds = []

	if len(review_list) > 0:
		for review in review_list:
			score = sentiment_score(review)
			all_sentiments.append(score)

	if len(all_sentiments) > 0:
		for sentiment_dict in all_sentiments:
			compound = sentiment_dict['compound']
			compounds.append(compound)

	if len(compounds) > 0:
		avg_sentiment = sum(compounds) / len(compounds)
	
	else:
		avg_sentiment = None

	return avg_sentiment


# def get_scores():
# 	if len(food_review_list) > 0:
# 		food_sentiment_score = round((get_sentiment(food_review_list))*150)
# 		if food_sentiment_score < 0:
# 			food_sentiment_score = 0
# 	else: 
# 		food_sentiment_score = 75
# 	if len(service_review_list) > 0:
# 		service_sentiment_score = round((get_sentiment(service_review_list))*150)
# 		if service_sentiment_score < 0:
# 			service_sentiment_score = 0
# 	else: 
# 		service_sentiment_score = 75
# 	if len(speed_review_list) > 0:
# 		speed_sentiment_score = round((get_sentiment(speed_review_list))*150)
# 		if speed_sentiment_score < 0:
# 			speed_sentiment_score = 0
# 	else: 
# 		speed_sentiment_score = 75
# 	if len(price_review_list) > 0:
# 		price_sentiment_score = round((get_sentiment(price_review_list))*150)
# 		if price_sentiment_score < 0:
# 			price_sentiment_score = 0
# 	else: 
# 		price_sentiment_score = 75
# 	if len(ambience_review_list) > 0:
# 		ambience_sentiment_score = round((get_sentiment(ambience_review_list))*150)
# 		if ambience_sentiment_score < 0:
# 			ambience_sentiment_score = 0
# 	else: 
# 		ambience_sentiment_score = 75
# 	if len(experience_review_list) > 0:
# 		experience_sentiment_score = round((get_sentiment(experience_review_list))*150)
# 		if experience_sentiment_score < 0:
# 			experience_sentiment_score = 0
# 	else: 
# 		experience_sentiment_score = 75
# 	return food_sentiment_score, service_sentiment_score, speed_sentiment_score, price_sentiment_score, ambience_sentiment_score, experience_sentiment_score

def get_scores():
	subspecreviews = [food_review_list, service_review_list, speed_review_list, price_review_list, ambience_review_list, experience_review_list]
	score_results = []
	for subjectreviews in subspecreviews:
		if len(subjectreviews) > 0:
			sub_sentiment_score = round((get_sentiment(subjectreviews))*150)
			if sub_sentiment_score < 0:
				sub_sentiment_score = 0
		else: 
			sub_sentiment_score = 75
			score_results.append(sub_sentiment_score)
	return score_results




extract_subject_related_words()
food_review_list, service_review_list, speed_review_list, price_review_list, ambience_review_list, experience_review_list = get_list_reviews_containing_subject()
subspecreviews = [food_review_list, service_review_list, speed_review_list, price_review_list, ambience_review_list, experience_review_list]
result = get_scores()
	
    
    # result = json.dumps([
# 			{ 'subject': 'Food', 'data1': food_sentiment_score, 'data2': 0, 'maxValue': 150 },
# 			{ 'subject': 'Service', 'data1': service_sentiment_score, 'data2': 0, 'maxValue': 150 },
# 			{ 'subject': 'Speed', 'data1': speed_sentiment_score, 'data2': 0, 'maxValue': 150 },
# 			{ 'subject': 'Price', 'data1': price_sentiment_score, 'data2': 0, 'maxValue': 150 },
# 			{ 'subject': 'Ambience', 'data1': ambience_sentiment_score, 'data2': 0, 'maxValue': 150},
# 			{ 'subject': 'Experience', 'data1': experience_sentiment_score, 'data2': 0, 'maxValue': 150}
# ])

# del [food_sentiment_score, service_sentiment_score, speed_sentiment_score, price_sentiment_score, ambience_sentiment_score, experience_sentiment_score]

# 	return result

In [ ]:

food_sentiment_score, service_sentiment_score, speed_sentiment_score, price_sentiment_score, ambience_sentiment_score, experience_sentiment_score = get_scores()

In [ ]:

#Thoughts on condensing function

# scoreslist = [food_sentiment_score, service_sentiment_score, speed_sentiment_score, price_sentiment_score, ambience_sentiment_score, experience_sentiment_score]
# for score in scoreslist:
#   viztype4 = {
#       'subject': 'Food', 'data1': scoreslist[0], 'data2': 0, 'maxValue': 150 },
#       'subject': 'Service', 'data1': scoreslist[1], 'data2': 0, 'maxValue': 150 },
#       'subject': 'Speed', 'data1': scoreslist[2], 'data2': 0, 'maxValue': 150 },
#       'subject': 'Price', 'data1': scoreslist[3], 'data2': 0, 'maxValue': 150 },     
#       'subject': 'Ambience', 'data1': scoreslist[4], 'data2': 0, 'maxValue': 150 },
#       'subject': 'Experience', 'data1': scoreslist[5], 'data2': 0, 'maxValue': 150 },   
#   }

In [ ]:
# #Sample
# [
#         { subject: 'Subject 1', data1: 45, data2: 70, maxValue: 150 },
#         { subject: 'Subject 2', data1: 75, data2: 95, maxValue: 150 },
#         { subject: 'Subject 3', data1: 20, data2: 50, maxValue: 150 },
#         { subject: 'Example Subject 4', data1: 65, data2: 85, maxValue: 150 },
#         { subject: 'Food', data1: 35, data2: 45, maxValue: 150}
# ]

In [ ]:
def get_scores():
  subspecreviews = [food_review_list, service_review_list, speed_review_list, price_review_list, ambience_review_list, experience_review_list]
  score_results = []
  for subjectreviews in subspecreviews:
    if len(subjectreviews) > 0:
      sub_sentiment_score = round((get_sentiment(subjectreviews))*150)
      if sub_sentiment_score < 0:
        sub_sentiment_score = 0
    else: 
      sub_sentiment_score = 75
    score_results.append(sub_sentiment_score)
  return score_results
[food_sentiment_score, service_sentiment_score, speed_sentiment_score, price_sentiment_score, ambience_sentiment_score, experience_sentiment_score] = get_scores()
food_sentiment_score

In [ ]:
experience_sentiment_score

In [ ]:
analytic = pd.DataFrame(df, columns = buisness_col)
analytic.head()